In [1]:
!pip install tensorflow-gpu


In [2]:
from keras.datasets import mnist
import json 
import dask 
import dask.bag as db
import pandas as pd
import numpy as np
import torch 
import tensorflow as tf
from torchvision.models import resnet18 as resnet
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from dask import delayed 
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout

Using TensorFlow backend.


In [3]:
print(tf.__version__)

2.1.0


In [0]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [0]:
def add_dim(x):
    n, h, w = x.shape
    return tf.reshape(x, shape=(n, h, w, 1)) 

In [0]:
x_train = add_dim(x_train) / 255
x_test = add_dim(x_test) / 255
y_train = tf.cast(y_train, dtype=tf.float64)
y_test = tf.cast(y_test, dtype=tf.float64)

In [0]:
class LeNet5(Model):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = Conv2D(filters=6, kernel_size=(5, 5), strides=(1, 1), 
                            padding='valid', kernel_initializer=glorot_uniform(0), 
                            activation='tanh')
        
        self.average_pooling1 = AveragePooling2D(strides=(2, 2), pool_size=(2, 2))

        self.conv2 = Conv2D(filters=16, kernel_size=(5, 5), strides=(1, 1), 
                            padding='valid', kernel_initializer=glorot_uniform(0), 
                            activation='tanh')
        
        self.average_pooling2 = AveragePooling2D(strides=(2, 2), pool_size=(2, 2))

        self.conv3 = Conv2D(filters=120, kernel_size=(5, 5), strides=(1, 1), 
                            padding='valid', kernel_initializer=glorot_uniform(0), 
                            activation='tanh')
        
        self.flatten = Flatten()
        self.dense1 = Dense(84, activation='tanh')
        self.dense2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.average_pooling1(x)

        x = self.conv2(x)
        x = self.average_pooling2(x)

        x = self.flatten(x)
        x = self.dense1(x)
        
        return self.dense2(x) 
        



In [0]:
model = LeNet5()
criterion = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(128)
test_loader = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(128)

In [0]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs=10):
    for i in range(num_epochs):
        losses = []
        accuracies = []
        for (i_step, el) in tqdm(enumerate(train_loader)):
            x = el[0]
            y = tf.reshape(el[1], shape=(-1, 1))
            with tf.GradientTape() as tape:
                with tf.device("GPU"):        
                    prediction = model(x)
                    loss = criterion(y, prediction)
                    
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            losses.append(loss)
        
        for (j_step, el_val) in enumerate(test_loader):
            x_val = el_val[0]
            y_val = el_val[1]

            with tf.GradientTape() as tape:
                with tf.device("GPU"):
                    prediction_val = model(x_val)
                    prediction_logits = tf.argmax(prediction_val, axis=1)
                    prediction_logits = tf.cast(prediction_logits, dtype=tf.float64)
                    correct_predictions = tf.math.reduce_mean(tf.cast(prediction_logits == y_val, dtype=tf.float64))

            accuracies.append(correct_predictions)

        accuracies = tf.Variable(accuracies)
        losses = tf.Variable(losses)
        print(f"Epoch: {i}, loss on train: {tf.reduce_mean(losses)}, correct_predictions{tf.reduce_mean(accuracies)}")





In [10]:
train_model(model, train_loader, test_loader, criterion, optimizer, 7)

469it [00:08, 52.12it/s]
0it [00:00, ?it/s]

Epoch: 0, loss on train: 0.3999886214733124, correct_predictions0.9473892405063291


469it [00:07, 59.66it/s]
0it [00:00, ?it/s]

Epoch: 1, loss on train: 0.14843501150608063, correct_predictions0.9675632911392406


469it [00:07, 54.92it/s]
0it [00:00, ?it/s]

...

Epoch: 5, loss on train: 0.050768766552209854, correct_predictions0.9847705696202531


469it [00:07, 62.09it/s]
0it [00:00, ?it/s]

Epoch: 6, loss on train: 0.044282495975494385, correct_predictions0.985067246835443


88it [00:01, 56.37it/s]


KeyboardInterrupt: ignored